# Описание
В данном ноутбуке проводится доразметка неразмеченного, предварительно предобработанного датасета unlabeled_train_name (см. ноутбук EDA+feature_engineering).

На первом этапе с помощью графовой структуры из category_tree определяются родительские категории. Затем с использованием библиотеки FAISS создаётся векторная база категорий.

Для каждого объекта из датасета осуществляется поиск топ-5 ближайших категорий по евклидову расстоянию в эмбеддинг-пространстве (по отношению к категориям из category_tree).

Далее найденные кандидаты ранжируются с помощью кросс-энкодера, и в качестве финальной метки выбирается топ-1 категория.

# Import

In [1]:
import pandas as pd
import numpy as np

import networkx as nx

from datasets import Dataset

import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

import faiss
from sentence_transformers import CrossEncoder

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to('cuda:0')

In [ ]:
reranker = CrossEncoder('DiTy/cross-encoder-russian-msmarco')

In [ ]:
df = pd.read_parquet('unlabeled_train_name.parquet')
category_tree = pd.read_csv('category_tree.csv')

In [3]:
df.head()

,name
0,комплект фиксации kict cu-332. для видеокарт
1,посудомоечная машина vestel df45e51w белый
2,уплотнитель для двери холодильника. подходит д...
3,кронштейн для телевизора 32-55 дюймов kaloc x4...
4,unimania держатель для телефона автомобильный ...


# Preprocessing

In [4]:
# заменяем NaN в parent_id на 0.0
category_tree['parent_id'] = category_tree['parent_id'].fillna(0)

# преобразование в значений в колонках
category_tree['parent_id'] = category_tree['parent_id'].astype(int)
category_tree['cat_name'] = category_tree['cat_name'].astype('category')
category_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896 entries, 0 to 1895
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   cat_id     1896 non-null   int64   
 1   parent_id  1896 non-null   int32   
 2   cat_name   1896 non-null   category
dtypes: category(1), int32(1), int64(1)
memory usage: 103.9 KB


In [5]:
# cоздаем граф (иерархию)
G = nx.DiGraph()
for _, row in category_tree.iterrows():
    G.add_edge(row['parent_id'], row['cat_id'])  # parent → child

terminators = [
    x for x in G.nodes() if G.out_degree(x) == 0 and G.in_degree(x) == 1
]  # ищем конечные узлы

print(terminators)
print(len(terminators))

[117, 152, 164, 169, 170, 173, 176, 177, 180, 182, 185, 188, 192, 196, 233, 235, 237, 277, 280, 283, 284, 287, 290, 327, 361, 390, 392, 404, 406, 417, 423, 438, 445, 446, 460, 465, 469, 1001, 1003, 1007, 1019, 1020, 1021, 1025, 1026, 1027, 1030, 1045, 1046, 1054, 1055, 1061, 1064, 1072, 1077, 1078, 1081, 1082, 1083, 1085, 1088, 1090, 1094, 1101, 1102, 1106, 1115, 1116, 1118, 1122, 1124, 1131, 1132, 1133, 1134, 1135, 1137, 1139, 1140, 1141, 1148, 1149, 1152, 1154, 1158, 1161, 1166, 1170, 1171, 1184, 1185, 1186, 1198, 1199, 1205, 1206, 1211, 1216, 1229, 1234, 1238, 1249, 1250, 1251, 1253, 1259, 1260, 1261, 1265, 1268, 1273, 1275, 1276, 1277, 1290, 1291, 1293, 1295, 1300, 1303, 1304, 1305, 1306, 1311, 1312, 1321, 1322, 1327, 1329, 1335, 1338, 1344, 1345, 1347, 1350, 1357, 1361, 1362, 1365, 1367, 1370, 1372, 1375, 1376, 1377, 1378, 1379, 1380, 1382, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1407, 1409, 1412, 1413, 1414, 1415, 1417, 1426, 1427, 1430, 1441, 1447, 1450, 1463

In [6]:
# выбираем конечные категории
category_tree = category_tree[category_tree['cat_id'].isin(terminators)]

In [7]:
category_tree.head()

,cat_id,parent_id,cat_name
38,117,1,Радиоуправляемые модели
72,152,4,Детская одежда и обувь
81,164,5,Парники и теплицы
84,169,6,"Макароны, крупы"
85,170,6,Растительное масло


In [8]:
# преобразуем данные в dataset
dataset = Dataset.from_pandas(df)
category_ds = Dataset.from_pandas(category_tree)

In [10]:
# получение эмбеддингов для неразмеченного датасета
def get_embeddings(texts):
    """
    Получает эмбеддинги для текста.
    """
    inputs = tokenizer(f"query: {texts['name']}",
                       padding=True,
                       truncation=True,
                       max_length=27,
                       return_tensors='pt')
    inputs = {key: val.to('cuda:0')
              for key, val in inputs.items()}  # отправляем на GPU
    
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.cpu().sum(dim=1)
    embeddings = F.normalize(embeddings, p=2, dim=1)
    texts['bert_emb'] = embeddings.squeeze()
    
    return texts


# применяем к датасету
dataset_emb = dataset.map(get_embeddings)

Map:   0%|          | 0/784742 [00:00<?, ? examples/s]

In [11]:
# получение эмбеддингов для датасета category_tree
def get_embeddings_cat(texts):
    """
    Получает эмбеддинги для текста.
    """
    inputs = tokenizer(f"query: {texts['cat_name']}",
                       padding=True,
                       truncation=True,
                       max_length=27,
                       return_tensors='pt')
    inputs = {key: val.to('cuda:0')
              for key, val in inputs.items()}  # отправляем на GPU

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.cpu().sum(dim=1)  # Берем CLS токен
    embeddings = F.normalize(embeddings, p=2, dim=1)
    texts['bert_emb'] = embeddings.squeeze()
    
    return texts


# применяем к датасету
category_embedding = category_ds.map(get_embeddings_cat)

Map:   0%|          | 0/1406 [00:00<?, ? examples/s]

In [12]:
# сохраняем обработанные датасеты
dataset_emb = pd.DataFrame(dataset_emb['bert_emb'])
dataset_emb.to_parquet('dataset_emb.parquet', index=False)

category_embedding = pd.DataFrame(category_embedding['bert_emb'])
category_embedding.to_parquet('category_embedding.parquet', index=False)

In [13]:
category_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.013414,-0.014273,-0.029773,-0.048221,0.036815,-0.029010,-0.024017,0.108831,0.058724,-0.015389,...,-0.014562,-0.012051,0.007828,-0.005488,-0.018361,0.007261,0.017712,-0.001841,-0.008097,0.026774
1,0.032827,-0.014384,-0.025819,-0.006944,0.036096,-0.002826,-0.009879,0.084608,0.043732,-0.015487,...,-0.003692,-0.043608,0.004786,-0.016799,0.004248,0.022676,0.001343,0.014661,-0.035101,-0.006278
2,0.026521,-0.006469,-0.018587,-0.018326,0.034383,-0.045831,-0.013413,0.091562,0.034955,-0.034436,...,-0.027361,-0.022528,0.018664,-0.008917,0.004040,0.047897,0.004496,-0.011304,-0.006231,0.014041
3,0.037698,-0.001712,-0.012433,-0.033278,0.047253,0.000388,-0.027752,0.091067,0.032933,-0.029515,...,-0.044412,-0.026803,0.019281,0.011100,-0.018829,0.032191,-0.018411,0.001348,-0.020426,0.010103
4,0.030515,0.000105,-0.001582,-0.040708,0.022216,-0.040541,-0.005007,0.113615,0.053629,-0.028242,...,0.000252,-0.028102,-0.016710,0.006648,-0.006319,0.030813,-0.001912,-0.014749,-0.028580,0.023760


In [14]:
# преобразование данных для fiass
dataset_emb[0] = dataset_emb[0].apply(lambda x: np.array(x, dtype='float32'))
category_embedding[0] = category_embedding[0].apply(lambda x: np.array(x, dtype='float32'))

# Marking

## Faiss

In [ ]:
def faiss_search(dataset, category_emb, category_tree):
    """
    Размечает датасет с помощью faiss.
    
    :param dataset: датасет с эмбеддингами объектов, которые нужно разметить.
    :param category_emb: словарь, где ключи - названия категорий, значения — их эмбеддинги.
    :param category_tree: датасет содержащий информацию о категориях.
    :return: список найденных меток категорий, 
            где каждая строка соответствует топ-5 категориям для одного объекта.
    """
    labels_df = []

    dimension = 1024
    index = faiss.IndexFlatL2(
        dimension)  # инициализирует FAISS индекс для поиска по L2 расстоянию

    # добавляем категории в FAISS
    embeddings = np.vstack(category_emb.values).astype(np.float32)

    index.add(embeddings)

    for i in tqdm(range(dataset.shape[0])):
        search = dataset.iloc[i].values.astype(np.float32).reshape(1, -1)

        D, I = index.search(search, 5)
        labels = category_tree.iloc[I.flatten(
        )]['cat_id'].values  # размечаем категории по category_tree

        labels_df.append(labels)

    return labels_df

In [22]:
# проводим доразметку
ldf = faiss_search(dataset_emb, category_embedding, category_tree)

100%|██████████| 784742/784742 [03:34<00:00, 3650.99it/s]


In [23]:
ldf[0:5]

[array([ 1409, 30977,  1293, 14407, 10421], dtype=int64),
 array([ 1376, 11292,  1401, 11293, 10496], dtype=int64),
 array([10493,  1399, 14027, 14025, 10492], dtype=int64),
 array([ 1238,  1304, 10118,  3610, 10127], dtype=int64),
 array([12290, 10004, 10028, 14126, 12322], dtype=int64)]

In [24]:
ldf = pd.DataFrame(
    pd.Series((v for v in ldf)),
    columns=['labels'])  # создаем датафрейм из полученных меток категорий

In [31]:
ldf.head(10)

,labels
0,"[1409, 30977, 1293, 14407, 10421]"
1,"[1376, 11292, 1401, 11293, 10496]"
2,"[10493, 1399, 14027, 14025, 10492]"
3,"[1238, 1304, 10118, 3610, 10127]"
4,"[12290, 10004, 10028, 14126, 12322]"
5,"[12278, 10024, 14134, 14317, 14216]"
6,"[3609, 1001, 10011, 3233, 1141]"
7,"[1327, 10084, 14389, 10081, 10165]"
8,"[10556, 10555, 10528, 10526, 1634]"
9,"[1306, 1064, 1081, 1085, 10153]"


In [56]:
# сохраняем датасет
ldf.to_parquet('f_cat.parquet', index=False)

## Reranker

In [28]:
# ранжируем cat_id reranker, получаем список топ-1 cat_id
num_cat_id_final = 1

# создаём словарь для быстрого поиска cat_id → cat_name
cat_dict = dict(zip(category_tree['cat_id'], category_tree['cat_name']))

# преобразуем каждый список id в список имен
relevant_cat_id = [[cat_dict[j] for j in i if j in cat_dict]
                   for i in ldf['labels']]

unlabeled_train_lst = df['name'].tolist()

new_relevant_cat_id = []
c = 0
for i, j in tqdm(
        zip(unlabeled_train_lst,
            relevant_cat_id), total=len(unlabeled_train_lst)
):  # проходит по названиям в неразмеченном датасете и названиям из топ-5 в размеченном
    relevant = reranker.rank(i, j)

    new_relevant_docs = []
    for k in relevant:
        new_relevant_docs.append(
            j[k['corpus_id']]
        )  # сопоставляет ответ reranker и список названий cat_id

    relevant_docs = new_relevant_docs
    relevant_docs = relevant_docs[:num_cat_id_final]
    search_value = relevant_docs[0]

    # получаем cat_id из названия
    for key, value in cat_dict.items():
        if value == search_value:
            new_relevant_cat_id.append(key)
            break

rerank_unlabeled_train_cat_id = pd.DataFrame({'label': new_relevant_cat_id})

100%|██████████| 784742/784742 [1:23:57<00:00, 155.79it/s]


In [29]:
# сохраняем датасет
rerank_unlabeled_train_cat_id.to_parquet('rerank_unlabeled_train_cat_id.parquet', index=False)

In [32]:
rerank_unlabeled_train_cat_id.head(10)

,label
0,1409
1,1376
2,10493
3,1238
4,12290
5,12278
6,1001
7,1327
8,10528
9,1306
